In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import os.path

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

from tensorflow import keras

from sklearn.metrics import r2_score

In [ ]:
image_dir = Path('/content/drive/MyDrive/Data/CNN PCA/source2')

In [ ]:
filepaths = pd.Series(list(image_dir.glob(r'**/*.png')), name='Filepath').astype(str)

In [ ]:
bpms = pd.Series(filepaths.apply(lambda x: (os.path.split(os.path.split(x)[1])[1])[:-4]), name='BPMS').astype(np.float64)

In [ ]:
images = pd.concat([filepaths, bpms], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [ ]:
image_df = images.sample(len(images), random_state = 1).reset_index(drop=True)
train_df, test_df = train_test_split(image_df, train_size= 0.7, shuffle= True, random_state=1)

In [ ]:
len(images)

739

In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split=0.4
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
)

train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='BPMS',
    target_size=(216, 144),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='BPMS',
    target_size=(216, 144),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='BPMS',
    target_size=(216, 144),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

Found 311 validated image filenames.
Found 206 validated image filenames.
Found 222 validated image filenames.


In [ ]:
base_model = keras.applications.ResNet50(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(216, 144, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(216, 144, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(1, activation='linear')(x)
model = keras.Model(inputs, outputs)

**Training #1**

In [ ]:
model.compile(
    optimizer='adam',
    loss='mae'
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/50
10/10 [==============================] - 138s 12s/step - loss: 79.6031 - val_loss: 77.0914
Epoch 2/50
10/10 [==============================] - 55s 6s/step - loss: 76.0513 - val_loss: 73.5556
Epoch 3/50
10/10 [==============================] - 55s 6s/step - loss: 72.5354 - val_loss: 70.0267
Epoch 4/50
10/10 [==============================] - 55s 6s/step - loss: 69.0783 - val_loss: 66.4966
Epoch 5/50
10/10 [==============================] - 55s 6s/step - loss: 65.5658 - val_loss: 62.9688
Epoch 6/50
10/10 [==============================] - 55s 6s/step - loss: 62.1055 - val_loss: 59.4329
Epoch 7/50
10/10 [==============================] - 55s 6s/step - loss: 58.5914 - val_loss: 55.9011
Epoch 8/50
10/10 [==============================] - 54s 6s/step - loss: 55.0772 - val_loss: 52.3628
Epoch 9/50
10/10 [==============================] - 55s 6s/step - loss: 51.5078 - val_loss: 48.8341
Epoch 10/50
10/10 [==============================] - 55s 6s/step - loss: 47.9852 - val_loss: 45.29

In [ ]:
from keras.models import model_from_json
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Saved model to disk


**Training #2**

In [ ]:
from keras.models import model_from_json
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.001)
loaded_model.compile(
    optimizer=opt,
    loss='mae'
)

history = loaded_model.fit(
    train_images,
    validation_data=val_images,
    epochs=20,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        )
    ]
)

Epoch 1/20
10/10 [==============================] - 63s 6s/step - loss: 11.1542 - val_loss: 11.4140
Epoch 2/20
10/10 [==============================] - 58s 6s/step - loss: 11.0896 - val_loss: 11.3761
Epoch 3/20
10/10 [==============================] - 57s 6s/step - loss: 11.0538 - val_loss: 11.3470
Epoch 4/20
10/10 [==============================] - 56s 6s/step - loss: 11.0264 - val_loss: 11.3261
Epoch 5/20
10/10 [==============================] - 56s 6s/step - loss: 10.9871 - val_loss: 11.3044
Epoch 6/20
10/10 [==============================] - 56s 6s/step - loss: 10.9644 - val_loss: 11.2486
Epoch 7/20
10/10 [==============================] - 54s 6s/step - loss: 10.9230 - val_loss: 11.2508
Epoch 8/20
10/10 [==============================] - 55s 6s/step - loss: 10.8894 - val_loss: 11.2475
Epoch 9/20
10/10 [==============================] - 55s 6s/step - loss: 10.8644 - val_loss: 11.2076
Epoch 10/20
10/10 [==============================] - 55s 6s/step - loss: 10.8377 - val_loss: 11.2288

In [ ]:
from keras.models import model_from_json
# serialize model to JSON
model_json = loaded_model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
loaded_model.save_weights("model.h5")
print("Saved model to disk")

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Saved model to disk


In [ ]:
predicted_bpm = np.squeeze(model.predict(test_images))
true_bpm  = test_images.labels

rmse = np.sqrt(model.evaluate(test_images, verbose=0))
print('Test RMSE: {:.5f}'.format(rmse))

r2 = r2_score(true_bpm, predicted_bpm)
print('Test R2 Score: {:.5f}'.format(r2))

Test RMSE: 3.15528
Test R2 Score: -0.09998


In [ ]:
max(predicted_bpm)

91.93422

In [ ]:
max(true_bpm)

146.52380952380952

In [ ]:
np.mean(np.abs(predicted_bpm-true_bpm))

9.955815647977412

In [ ]:
test_image_dir = Path('/content/drive/MyDrive/Data/CNN ICA/source2/v8')
test_filepaths = pd.Series(list(test_image_dir.glob(r'**/*.png')), name='Filepath').astype(str)
test_bpms = pd.Series(test_filepaths.apply(lambda x: (os.path.split(os.path.split(x)[1])[1])[:-4]), name='BPMS').astype(np.float64)
test_images = pd.concat([test_filepaths, test_bpms], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  


In [ ]:
len(test_images)

105

In [ ]:
test_image_df = test_images.sample(105, random_state = 1).reset_index(drop=True)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
)
test_images_flow = test_generator.flow_from_dataframe(
    dataframe=test_image_df,
    x_col='Filepath',
    y_col='BPMS',
    target_size=(216, 144),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

Found 105 validated image filenames.


In [ ]:
predicted_bpm = np.squeeze(model.predict(test_images_flow))
true_bpm  = test_images_flow.labels

In [ ]:
np.mean(np.abs(predicted_bpm-true_bpm))

11.881046956791165

In [ ]:
predicted_bpm

array([83.48346 , 77.90767 , 80.190125, 77.94206 , 79.538536, 77.562744,
       78.28451 , 78.23456 , 77.13828 , 79.25375 , 77.01563 , 80.40961 ,
       79.86366 , 80.275955, 76.150314, 78.73187 , 77.77386 , 76.98876 ,
       73.69743 , 81.71245 , 80.45623 , 76.50362 , 77.3042  , 83.04311 ,
       78.75156 , 79.16561 , 78.438644, 74.09755 , 80.0482  , 73.57709 ,
       80.034744, 77.043274, 78.40629 , 82.76926 , 78.19068 , 83.15059 ,
       76.942276, 76.55005 , 75.27224 , 77.36964 , 77.62869 , 80.35218 ,
       78.77774 , 77.920395, 78.15559 , 84.304535, 78.11423 , 77.53462 ,
       78.73544 , 77.72739 , 72.65702 , 81.47362 , 83.08504 , 75.06857 ,
       75.72966 , 74.88263 , 75.49421 , 74.04792 , 77.447624, 77.577736,
       77.8412  , 80.02667 , 75.88284 , 74.940605, 78.99249 , 76.48762 ,
       84.45316 , 81.0774  , 80.90716 , 77.308914, 76.19949 , 79.09288 ,
       75.671616, 83.09895 , 78.48091 , 79.85566 , 80.06098 , 82.6839  ,
       78.22043 , 88.49462 , 88.07305 , 76.88611 , 

In [ ]:
for i in range(1,3):  
  test_image_dir = Path('/content/drive/MyDrive/Data/CNN ICA/source3/v{}'.format(i))
  test_filepaths = pd.Series(list(test_image_dir.glob(r'**/*.png')), name='Filepath').astype(str)
  test_bpms = pd.Series(test_filepaths.apply(lambda x: (os.path.split(os.path.split(x)[1])[1])[:-4]), name='BPMS').astype(np.float64)
  test_images = pd.concat([test_filepaths, test_bpms], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)
  test_image_df = test_images.sample(len(test_images), random_state = 1).reset_index(drop=True)
  test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
  )
  test_images_flow = test_generator.flow_from_dataframe(
      dataframe=test_image_df,
      x_col='Filepath',
      y_col='BPMS',
      target_size=(216, 144),
      color_mode='rgb',
      class_mode='raw',
      batch_size=32,
      shuffle=False
  )
  predicted_bpm = np.squeeze(model.predict(test_images_flow))
  true_bpm  = test_images_flow.labels
  print("v" + str(i) + ": " + str(np.mean(np.abs(predicted_bpm-true_bpm))))

Found 104 validated image filenames.
v1: 8.637015840838323
Found 106 validated image filenames.
v2: 8.715689803338698


In [ ]:
test_image_dir = Path('/content/drive/MyDrive/Data/CNN ICA/source3/v3-v5')
test_filepaths = pd.Series(list(test_image_dir.glob(r'**/*.png')), name='Filepath').astype(str)
test_bpms = pd.Series(test_filepaths.apply(lambda x: (os.path.split(os.path.split(x)[1])[1])[:-4]), name='BPMS').astype(np.float64)
test_images = pd.concat([test_filepaths, test_bpms], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)
test_image_df = test_images.sample(len(test_images), random_state = 1).reset_index(drop=True)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
  )
test_images_flow = test_generator.flow_from_dataframe(
      dataframe=test_image_df,
      x_col='Filepath',
      y_col='BPMS',
      target_size=(216, 144),
      color_mode='rgb',
      class_mode='raw',
      batch_size=32,
      shuffle=False
  )
predicted_bpm = np.squeeze(model.predict(test_images_flow))
true_bpm  = test_images_flow.labels
print("v" + str(i) + ": " + str(np.mean(np.abs(predicted_bpm-true_bpm))))

Found 210 validated image filenames.
v2: 8.365731668721322


In [ ]:
for i in range(4,5):  
  test_image_dir = Path('/content/drive/MyDrive/Data/CNN ICA/source3/v{}'.format(i))
  test_filepaths = pd.Series(list(test_image_dir.glob(r'**/*.png')), name='Filepath').astype(str)
  test_bpms = pd.Series(test_filepaths.apply(lambda x: (os.path.split(os.path.split(x)[1])[1])[:-4]), name='BPMS').astype(np.float64)
  test_images = pd.concat([test_filepaths, test_bpms], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)
  test_image_df = test_images.sample(len(test_images), random_state = 1).reset_index(drop=True)
  test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
  )
  test_images_flow = test_generator.flow_from_dataframe(
      dataframe=test_image_df,
      x_col='Filepath',
      y_col='BPMS',
      target_size=(216, 144),
      color_mode='rgb',
      class_mode='raw',
      batch_size=32,
      shuffle=False
  )
  predicted_bpm = np.squeeze(model.predict(test_images_flow))
  true_bpm  = test_images_flow.labels
  print("v" + str(i) + ": " + str(np.mean(np.abs(predicted_bpm-true_bpm))))

Found 105 validated image filenames.
v4: 8.003995001853502
